In [ ]:
import requests
from bs4 import BeautifulSoup
import sys
import json



In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
}

In [ ]:
# Set to track the URLs (unique identifiers) of jobs already added
seen_urls = set()


In [ ]:
def fetch_body_content(url, headers):
    try:
        response = requests.get(url=url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        body_content_div = soup.find('div', class_="body-content")
        
        if body_content_div:
            # Getting the text within the body-content class
            return body_content_div.get_text(strip=True)  # or .prettify() for HTML
    except requests.RequestException:
        print(f"Failed to retrieve body content from {url}")


    return None

    

def scrape_sector(url, headers, seen_urls):
    # Initialise a list to hold the scraped data
    job_listings = []
    
    try:
        response = requests.get(url=url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        a_tags = soup.find_all('a', class_="tw-block tw-text-base tw-font-semibold")

        
        for tag in a_tags:
            job_url = tag['href']

            if job_url in seen_urls:
                continue

            body_content = fetch_body_content(url=job_url, headers=headers)
            body_content_fetched = body_content is not None

            if body_content_fetched:

                job = {
                    'url': job_url,
                    'title': tag.get_text(strip=True),
                }
                if 'title' in tag.attrs:
                    job['Description'] = tag['title']
    
                # Extract additional info from the <ul>
                ul = tag.find_next_sibling('ul', class_="tw-pl-0 tw-mt-0 tw-mb-0 tw-space-y-1.5 tw-text-xs tw-font-medium tw-py-2 tw-text-gray-500 tw-list-none")
                if ul:
                    for li in ul.find_all('li'):
                        key = li.find('span').get_text(strip=True).rstrip(':')
                        value = li.get_text(strip=True).replace(key + ':', '').strip()
                        job[key] = value  
                job['body_content'] = body_content
    
    
                job_listings.append(job)
                # Add the URL to the set of seen URLs
                seen_urls.add(job_url)
               

    except requests.exceptions.RequestException as e:
        print(f"Request failed for {url}: {e}")
    except Exception as e:
        print(f"An error occurred for {url}: {e}")
    return job_listings




In [ ]:
# List of URLs to scrape
graduate_job_sectors_urls = [
    "https://www.gradcracker.com/search/aerospace/engineering-graduate-jobs", 
    "https://www.gradcracker.com/search/chemical-process/engineering-graduate-jobs",
    "https://www.gradcracker.com/search/civil-building/engineering-graduate-jobs",
    "https://www.gradcracker.com/search/computing-technology/graduate-jobs",
    "https://www.gradcracker.com/search/electronic-electrical/engineering-graduate-jobs",
    "https://www.gradcracker.com/search/maths-business/graduate-jobs",
    "https://www.gradcracker.com/search/mechanical-manufacturing/engineering-graduate-jobs",
    "https://www.gradcracker.com/search/science/graduate-jobs"]

In [ ]:
# Initialise the graduate_listings list that will hold the graduate job listings scraped
# This will be done sector by sector over time
graduate_listings = [] 

In [ ]:
graduate_listings.extend(scrape_sector(graduate_job_sectors_urls[0], headers=headers, seen_urls=seen_urls)) 

In [ ]:
graduate_listings.extend(scrape_sector(graduate_job_sectors_urls[1], headers=headers, seen_urls=seen_urls)) 

In [ ]:
graduate_listings.extend(scrape_sector(graduate_job_sectors_urls[2], headers=headers, seen_urls=seen_urls))

In [ ]:
graduate_listings.extend(scrape_sector(graduate_job_sectors_urls[3], headers=headers, seen_urls=seen_urls))

In [ ]:
graduate_listings.extend(scrape_sector(graduate_job_sectors_urls[4], headers=headers, seen_urls=seen_urls))

In [ ]:
graduate_listings.extend(scrape_sector(graduate_job_sectors_urls[5], headers=headers, seen_urls=seen_urls))

In [ ]:
graduate_listings.extend(scrape_sector(graduate_job_sectors_urls[6], headers=headers, seen_urls=seen_urls))

In [ ]:
graduate_listings.extend(scrape_sector(graduate_job_sectors_urls[7], headers=headers, seen_urls=seen_urls))

In [ ]:
graduate_listings

In [ ]:
with open('graduate_listings_full.json', 'w') as json_file:
    json.dump(graduate_listings, json_file, indent=4)

In [ ]:
palcement_internship_sector_urls = [
    "https://www.gradcracker.com/search/aerospace/engineering-work-placements-internships",
    "https://www.gradcracker.com/search/chemical-process/engineering-work-placements-internships",
    "https://www.gradcracker.com/search/civil-building/engineering-work-placements-internships",
    "https://www.gradcracker.com/search/computing-technology/work-placements-internships",
    "https://www.gradcracker.com/search/electronic-electrical/engineering-work-placements-internships",
    "https://www.gradcracker.com/search/maths-business/work-placements-internships",
    "https://www.gradcracker.com/search/mechanical-manufacturing/engineering-work-placements-internships",
    "https://www.gradcracker.com/search/science/work-placements-internships"
]

In [ ]:
# Initialise the placement_internship_list as previously done for graduate_listings
placement_internship_listings = []

In [ ]:
placement_internship_listings.extend(scrape_sector(palcement_internship_sector_urls[0], headers=headers, seen_urls=seen_urls))

In [ ]:
placement_internship_listings.extend(scrape_sector(palcement_internship_sector_urls[1], headers=headers, seen_urls=seen_urls))

In [ ]:
placement_internship_listings.extend(scrape_sector(palcement_internship_sector_urls[2], headers=headers, seen_urls=seen_urls))

In [ ]:
placement_internship_listings.extend(scrape_sector(palcement_internship_sector_urls[3], headers=headers, seen_urls=seen_urls))

In [ ]:
placement_internship_listings.extend(scrape_sector(palcement_internship_sector_urls[4], headers=headers, seen_urls=seen_urls))

In [ ]:
placement_internship_listings.extend(scrape_sector(palcement_internship_sector_urls[5], headers=headers, seen_urls=seen_urls))

In [ ]:
placement_internship_listings.extend(scrape_sector(palcement_internship_sector_urls[6], headers=headers, seen_urls=seen_urls))

In [ ]:
placement_internship_listings.extend(scrape_sector(palcement_internship_sector_urls[7], headers=headers, seen_urls=seen_urls))

In [ ]:
import json 

with open('placement_internship_listings_full.json', 'w') as json_file:
    json.dump(placement_internship_listings, json_file, indent=4)